### Fetching Finance Data 

In [2]:
!pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --------------------------- ------------ 2.1/3.0 MB 11.8 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 10.1 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15618 sha256=38654fe6d8432d23b5d909157408ea9da7897b0fa28464a9aa78c9245271f279
  Stored in directory: c:\user

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [3]:
import yfinance as yf
import requests
import pandas as pd

In [12]:
import yfinance as yf
import pandas as pd

# Choose stock and date range
ticker = "TSLA"
start_date = "2023-01-01"
end_date = "2025-01-01"
tsla_data = yf.download(ticker, start=start_date, end=end_date, interval="1d")

# Reset index so Date becomes a column
tsla_data = tsla_data.reset_index()

# Show first 5 rows
tsla_data.head()

C:\Users\Niyati\AppData\Local\Temp\ipykernel_16252\2352384642.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tsla_data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,TSLA,TSLA,TSLA,TSLA,TSLA
0,2023-01-03,108.099998,118.800003,104.639999,118.470001,231402800
1,2023-01-04,113.639999,114.589996,107.519997,109.110001,180389000
2,2023-01-05,110.339996,111.750000,107.160004,110.510002,157986300
3,2023-01-06,113.059998,114.389999,101.809998,103.000000,220911100
4,2023-01-09,119.769997,123.519997,117.110001,118.959999,190284000


### Fetching News Data 

In [15]:
from newsapi import NewsApiClient
API_KEY = "f0f20f288bb741398777cce6f6988bc9"
newsapi = NewsApiClient(api_key=API_KEY)
print(newsapi)

In [30]:
from_date = "2025-10-17"
to_date   = "2025-11-17"

topics = {
    "tesla": 'Tesla OR TSLA',
    "musk": '"Elon Musk"',
    "ev": '"electric vehicles" OR "EV market"',
    "policy": '"EV tax credit" OR "climate policy"'
}

In [31]:
def get_news_simple(query, from_date, to_date, api_key):
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "from": from_date,
        "to": to_date,
        "language": "en",
        "sortBy": "publishedAt",
        "pageSize": 50,   # first 50 articles
        "page": 1,
        "apiKey": api_key,
    }
    r = requests.get(url, params=params)
    data = r.json()
    if data.get("status") != "ok":
        print("Error:", data)
        return pd.DataFrame()
    df = pd.json_normalize(data["articles"])
    return df

# collect all topics
dfs = []
for name, query in topics.items():
    df = get_news_simple(query, from_date, to_date, API_KEY)
    if not df.empty:
        df["topic"] = name
        dfs.append(df)
        print(name, "articles:", len(df))

# combine
news_all = pd.concat(dfs, ignore_index=True)

# add date column
news_all["publishedAt"] = pd.to_datetime(news_all["publishedAt"])
news_all["date"] = news_all["publishedAt"].dt.date
news_all["date"] = pd.to_datetime(news_all["date"])

# keep only useful columns
news_small = news_all[["date", "topic", "title", "description", "content", "url"]]

news_small.head()

tesla articles: 50
musk articles: 48
ev articles: 48
policy articles: 45


,date,topic,title,description,content,url
0,2025-11-16,tesla,Elon Musk Says It's 'Mindblowing' How Much Mon...,In response to Alphabet Inc.‘s (NASDAQ:GOOGL) ...,In response to Alphabet Inc.s (NASDAQ:GOOGL) (...,https://biztoc.com/x/529acb6cef17c023
1,2025-11-16,tesla,7 Online Resume Builders Designers Actually Rate,Designers juggle two audiences in every résumé...,Designers juggle two audiences in every résumé...,https://www.designer-daily.com/online-resume-b...
2,2025-11-16,tesla,"Peruvian oil firm Petroperu reshuffles board, ...",,"{ window.open(this.href, '_blank'); }, 200); r...",https://biztoc.com/x/ea12c1d0b40f3c59
3,2025-11-16,tesla,A cat's killing by a Waymo robotaxi sparked ou...,,"{ window.open(this.href, '_blank'); }, 200); r...",https://biztoc.com/x/2ed90d63f7074039
4,2025-11-16,tesla,No. 3 Texas A&M pulls off biggest comeback in ...,,"{ window.open(this.href, '_blank'); }, 200); r...",https://biztoc.com/x/b82a6a97cd7ae8c4


In [38]:
news_small["text"] = ((news_small["title"].fillna("")) + " " +(news_small["description"].fillna("")))
news_small[["topic", "text"]].head()

C:\Users\Niyati\AppData\Local\Temp\ipykernel_16252\1056424339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_small["text"] = ((news_small["title"].fillna("")) + " " +(news_small["description"].fillna("")))


,topic,text
0,tesla,Elon Musk Says It's 'Mindblowing' How Much Mon...
1,tesla,7 Online Resume Builders Designers Actually Ra...
2,tesla,"Peruvian oil firm Petroperu reshuffles board, ..."
3,tesla,A cat's killing by a Waymo robotaxi sparked ou...
4,tesla,No. 3 Texas A&M pulls off biggest comeback in ...


In [40]:
news_small.shape

(191, 7)

##### Install and run FinBert Sentiment

In [35]:
!pip install transformers torch --quiet

In [36]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Load FinBERT model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

C:\Users\Niyati\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Niyati\.cache\huggingface\hub\models--ProsusAI--finbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [41]:
def get_sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits.detach().numpy()[0]
    probs = np.exp(logits) / np.sum(np.exp(logits))
    positive, neutral, negative = probs
    return positive - negative  # sentiment score

news_small["sentiment_score"] = news_small["text"].apply(get_sentiment_score)

C:\Users\Niyati\AppData\Local\Temp\ipykernel_16252\2646121949.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_small["sentiment_score"] = news_small["text"].apply(get_sentiment_score)
